## ทดลองทำการ Evaluate


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# สมมุติว่ามี Ground Truth และ Predicted Text
ground_truth = "กรุงเทพมหานคร"
predicted_text = "กรุงเทหมาหา"

# สร้าง list ของตัวอักษรจาก Ground Truth และ Predicted Text
ground_truth_chars = list(ground_truth)
predicted_chars = list(predicted_text)

# เพิ่มความยาวของ list ให้ตรงกัน
max_len = max(len(ground_truth_chars), len(predicted_chars))
ground_truth_chars = ground_truth_chars + [''] * (max_len - len(ground_truth_chars))
predicted_chars = predicted_chars + [''] * (max_len - len(predicted_chars))

# สร้าง confusion matrix
cm = confusion_matrix(ground_truth_chars, predicted_chars, labels=np.unique(ground_truth_chars + predicted_chars))

# แสดง confusion matrix
print("Confusion Matrix:")
print(cm)

# แสดงผล classification report
print("Classification Report:")
print(classification_report(ground_truth_chars, predicted_chars))


### ทดลอง 29 Aug

พบว่ามีช่องว่าง ผลเลยเพี้ยน

In [10]:
import pytesseract
from PIL import Image
from datetime import datetime

# ตั้งค่าเส้นทางไปยัง Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'

# Path to the image file
image_path = '/Users/gunnviryasiri/Desktop/ocr/testfile/file1.jpg'

# Path to the reference file
reference_text_file = '/Users/gunnviryasiri/Desktop/ocr/results/FORTEST.txt'

# สร้างชื่อไฟล์จากวันที่และเวลา
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
output_text_file = f'/Users/gunnviryasiri/Desktop/ocr/results/ocr_output_{current_time}.txt'

# Load the image
image = Image.open(image_path)

# Convert the image to grayscale
gray = image.convert('L')

# Perform OCR on the image using Thai and English languages
custom_config = r'--tessdata-dir "/Users/gunnviryasiri/pythainlp-data/tessdata" --oem 3 --psm 6'
text = pytesseract.image_to_string(gray, config=custom_config, lang='tha+eng')

# Save the extracted text to a file
with open(output_text_file, 'w', encoding='utf-8') as f:
    f.write(text)

# อ่านข้อความที่ถูกต้องจากไฟล์ reference
with open(reference_text_file, 'r', encoding='utf-8') as f:
    correct_text_lines = f.readlines()

# อ่านข้อความที่ OCR มาได้
with open(output_text_file, 'r', encoding='utf-8') as f:
    ocr_text_lines = f.readlines()

# เปรียบเทียบข้อความทีละบรรทัด
tp = fp = fn = 0

for correct_line, ocr_line in zip(correct_text_lines, ocr_text_lines):
    correct_words = correct_line.strip().split()
    ocr_words = ocr_line.strip().split()

    # นับจำนวนคำที่ match กัน
    for word in correct_words:
        if word in ocr_words:
            tp += 1  # True Positive
            ocr_words.remove(word)  # เอาคำที่ match ออกเพื่อลดการนับซ้ำ
        else:
            fn += 1  # False Negative: คำที่ขาดหายไป

    # คำที่เหลือใน ocr_words คือตัว False Positive: คำที่เกินมา
    fp += len(ocr_words)

# แสดงผลลัพธ์ Confusion Matrix
print(f"True Positive (TP): {tp}")
print(f"False Positive (FP): {fp}")
print(f"False Negative (FN): {fn}")

accuracy = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')

print(f'Text extracted, cleaned, and saved to {output_text_file}')


True Positive (TP): 2
False Positive (FP): 1705
False Negative (FN): 33
Accuracy: 0.00
Precision: 0.00
Recall: 0.06
Text extracted, cleaned, and saved to /Users/gunnviryasiri/Desktop/ocr/results/ocr_output_20240829_200631.txt


In [5]:
import re

# ฟังก์ชันสำหรับลบช่องว่างที่ไม่จำเป็น
def remove_unnecessary_spaces(text):
    # ลบช่องว่างระหว่างตัวอักษรไทยที่ไม่ควรมีช่องว่าง
    text = re.sub(r'\s+', '', text)
    # ลบช่องว่างระหว่างอักษรไทยที่มีสระหรือตัวสะกด
    text = re.sub(r'([ก-ฮ])\s+([่-๋็-์])', r'\1\2', text)
    # ลบช่องว่างระหว่างตัวเลข
    text = re.sub(r'(\d)\s+(\d)', r'\1\2', text)
    return text

# ตัวอย่างการใช้งาน
ocr_text = "ท ี ่ ก พ ท ๐ ๕ /# ๑ ๕ ต ๒ ส ํ า น ั ก ง า น ก า ร บ ิ น พ ล เร ื อ น แห ่ ง ป ร ะ เท ศ ไท ย"
cleaned_text = remove_unnecessary_spaces(ocr_text)
print(cleaned_text)


ที่กพท๐๕/#๑๕ต๒สํานักงานการบินพลเรือนแห่งประเทศไทย


แก้ไม่ได้

In [9]:
import pytesseract
from PIL import Image
from datetime import datetime
import re

# ตั้งค่าเส้นทางไปยัง Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'

# Path to the image file
image_path = '/Users/gunnviryasiri/Desktop/ocr/testfile/file1.jpg'

# Path to the reference file
reference_text_file = '/Users/gunnviryasiri/Desktop/ocr/results/FORTEST.txt'

# สร้างชื่อไฟล์จากวันที่และเวลา
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
output_text_file = f'/Users/gunnviryasiri/Desktop/ocr/results/ocr_output_{current_time}.txt'

# Load the image
image = Image.open(image_path)

# Convert the image to grayscale
gray = image.convert('L')

# Perform OCR on the image using Thai and English languages
custom_config = r'--tessdata-dir "/Users/gunnviryasiri/pythainlp-data/tessdata" --oem 3 --psm 6'
text = pytesseract.image_to_string(gray, config=custom_config, lang='tha+eng')

# ฟังก์ชันสำหรับการประมวลผลข้อความหลังการ OCR
def post_process(text):
    # รวมคำที่มีช่องว่างเกิน
    text = re.sub(r'([ก-ฮa-zA-Z0-9])\s+([ก-ฮa-zA-Z0-9])', r'\1 \2', text)
    
    # ลบช่องว่างที่เกินที่ท้ายบรรทัด
    text = re.sub(r'\s+$', '', text)
    
    # แทนที่ช่องว่างที่เกินทั้งหมดในเอกสารด้วยช่องว่างเดียว
    text = re.sub(r'\s+', ' ', text)
    
    return text

# Post-process the OCR result
text = post_process(text)

# Save the extracted text to a file
with open(output_text_file, 'w', encoding='utf-8') as f:
    f.write(text)

# อ่านข้อความที่ถูกต้องจากไฟล์ reference
with open(reference_text_file, 'r', encoding='utf-8') as f:
    correct_text_lines = f.readlines()

# อ่านข้อความที่ OCR มาได้
with open(output_text_file, 'r', encoding='utf-8') as f:
    ocr_text_lines = f.readlines()

# เปรียบเทียบข้อความทีละบรรทัด
tp = fp = fn = 0

for correct_line, ocr_line in zip(correct_text_lines, ocr_text_lines):
    correct_words = set(correct_line.strip().split())
    ocr_words = set(ocr_line.strip().split())

    # คำนวณ True Positive, False Positive และ False Negative
    tp += len(correct_words.intersection(ocr_words))
    fp += len(ocr_words - correct_words)
    fn += len(correct_words - ocr_words)

# แสดงผลลัพธ์ Confusion Matrix
print(f"True Positive (TP): {tp}")
print(f"False Positive (FP): {fp}")
print(f"False Negative (FN): {fn}")

accuracy = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')

print(f'Text extracted, cleaned, and saved to {output_text_file}')


True Positive (TP): 1
False Positive (FP): 154
False Negative (FN): 0
Accuracy: 0.01
Precision: 0.01
Recall: 1.00
Text extracted, cleaned, and saved to /Users/gunnviryasiri/Desktop/ocr/results/ocr_output_20240829_200437.txt


## 12 sep
- ทำการเตรียม dataset น้อยๆ เพื่อเตรียม bipartite

- ข้อมูล แบบ shift หน้า, หลัง , เว้น2ช่อง เว้นช่องเดียว 

- ถ้าหากทำเสร็จก็ไปข้ามเรื่อง pre,post processings ได้แล้ว

In [ ]:
import pytesseract
from PIL import Image
from datetime import datetime
from pythainlp.corpus import thai_words
from pythainlp.tokenize import word_tokenize
import nltk
from nltk.corpus import words
import difflib
from difflib import SequenceMatcher

nltk.download('words', quiet=True)

# กำหนด path ของ Tesseract
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'

# ฟังก์ชันคำนวณ Similarity ระหว่างสองคำ
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# ฟังก์ชันหาการจับคู่ที่ดีที่สุดแบบ Global
def find_best_matching(reference, ocr):
    ref_chars = list(reference)
    ocr_chars = list(ocr)
    
    best_matching = []
    for i, ref_char in enumerate(ref_chars):
        best_match = None
        best_sim = 0
        for j, ocr_char in enumerate(ocr_chars):
            sim = similarity(ref_char, ocr_char)
            if sim > best_sim:
                best_sim = sim
                best_match = (i, j)
        
        if best_match:
            best_matching.append(best_match)
    
    return best_matching

# ฟังก์ชันสำหรับการ Post-Processing
def post_process_matching(reference, ocr, matching, accuracy_threshold=0.7):
    matched_chars = ['_'] * len(reference)
    correct_chars = 0
    error_details = []
    
    for ref_index, ocr_index in matching:
        if ref_index < len(reference) and ocr_index < len(ocr):
            if reference[ref_index] == ocr[ocr_index]:
                matched_chars[ref_index] = ocr[ocr_index]
                correct_chars += 1
            else:
                error_details.append(f"Position {ref_index + 1}: Expected '{reference[ref_index]}', Got '{ocr[ocr_index]}'")
    
    # Handle missing and extra characters
    if len(ocr) < len(reference):
        for i in range(len(ocr), len(reference)):
            error_details.append(f"Position {i + 1}: Expected '{reference[i]}', Got ' ' (missing)")
    elif len(ocr) > len(reference):
        for i in range(len(reference), len(ocr)):
            error_details.append(f"Position {i + 1}: Expected ' ' (extra), Got '{ocr[i]}'")
    
    accuracy = correct_chars / len(reference) if len(reference) > 0 else 0
    
    if accuracy >= accuracy_threshold:
        return "".join(matched_chars), accuracy, error_details
    else:
        return "".join(matched_chars), accuracy, error_details

# เริ่มกระบวนการ OCR
image_path = '/Users/gunnviryasiri/Desktop/ocr/testfile/file1.jpg'
reference_text_file = '/Users/gunnviryasiri/Desktop/ocr/results/FORTEST.txt'

# โหลดและประมวลผลรูปภาพ
image = Image.open(image_path)
gray_image = image.convert('L')

custom_config = r'--tessdata-dir "/Users/gunnviryasiri/pythainlp-data/tessdata" --oem 3 --psm 6'

ocr_text = pytesseract.image_to_string(gray_image, config=custom_config, lang='tha+eng')

# ลบเฉพาะบรรทัดว่าง
processed_ocr_text = "\n".join([line for line in ocr_text.splitlines() if line.strip()])

# บันทึกผลลัพธ์ OCR ลงไฟล์
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
output_text_file = f'/Users/gunnviryasiri/Desktop/ocr/results/ocr_output_{current_time}.txt'
with open(output_text_file, 'w', encoding='utf-8') as f:
    f.write(processed_ocr_text)

# โหลดข้อมูลต้นฉบับ
with open(reference_text_file, 'r', encoding='utf-8') as f:
    reference_text = f.read().replace("\n", "")

# ลบ newlines ในข้อความ OCR ที่ประมวลผลแล้ว
processed_ocr_text = processed_ocr_text.replace("\n", "")

# หาการจับคู่ที่ดีที่สุดแบบ Global
best_matching = find_best_matching(reference_text, processed_ocr_text)

# ทำการ Post-Processing โดยใช้ Global Match
processed_text, final_accuracy, error_details = post_process_matching(reference_text, processed_ocr_text, best_matching)

# คำนวณตัวชี้วัดผลลัพธ์สุดท้าย
total_chars = len(reference_text)
correct_chars = processed_text.count('_')  # เปลี่ยนตรงนี้เป็นการนับจำนวน underscore
incorrect_chars = total_chars - correct_chars

precision = (correct_chars / (correct_chars + incorrect_chars)) * 100 if (correct_chars + incorrect_chars) > 0 else 0
recall = (correct_chars / total_chars) * 100 if total_chars > 0 else 0

# แสดงผลลัพธ์
print("=== Confusion Matrix ===")
print(f"Total Characters in Reference Text: {total_chars}")
print(f"Correct Characters: {correct_chars}")
print(f"Incorrect Characters: {incorrect_chars}\n")

print("=== Metrics ===")
print(f"Final Accuracy: {final_accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}%")
print(f"Recall: {recall:.2f}%\n")

print("=== Character Error Details ===")
for detail in error_details:
    print(detail)

print("=== Processed Text ===")
print(processed_text)

print(f"\nText extracted, cleaned, and saved to {output_text_file}")
